In [1]:
# Task A - Montion Caputure & Visualization
# Name: Dongyi Guo
# Student ID: 470033070
# UniKey: dguo0006

In [3]:
#Headers:
import os
import sys
import cv2
import math
import random
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [4]:
# Appendix
def arrowdraw(img_draw, x1, y1, x2, y2):
    radians = math.atan2(x1-x2, y2-y1)
    x11 = 0
    y11 = 0
    x12 = -5
    y12 = -5

    u11 = 0
    v11 = 0
    u12 = 5
    v12 = -5
    
    x11_ = x11*math.cos(radians) - y11*math.sin(radians) + x2
    y11_ = x11*math.sin(radians) + y11*math.cos(radians) + y2

    x12_ = x12 * math.cos(radians) - y12 * math.sin(radians) + x2
    y12_ = x12 * math.sin(radians) + y12 * math.cos(radians) + y2
    
    u11_ = u11 * math.cos(radians) - v11 * math.sin(radians) + x2
    v11_ = u11 * math.sin(radians) + v11 * math.cos(radians) + y2

    u12_ = u12 * math.cos(radians) - v12 * math.sin(radians) + x2
    v12_ = u12 * math.sin(radians) + v12 * math.cos(radians) + y2
    img_draw = cv2.line(img_draw, (x1, y1), (x2, y2), (0, 255, 255), 2)
    img_draw = cv2.line(img_draw, (int(x11_), int(y11_)), (int(x12_), int(y12_)),
                        (0, 255, 255), 2)
    img_draw = cv2.line(img_draw, (int(u11_), int(v11_)), (int(u12_), int(v12_)),
                        (0, 255, 255), 2)
    
    return img_draw

In [5]:
def create_dir(dir_):
    if not os.path.exists(dir_):
        os.mkdir(dir_)
    else:
        print(dir_ + " exists")
frame_save_path = './frames/'
frame_num = 749

In [6]:
def calc_ssd(a,b):
    ssd = np.sum((a-b)**2)
    return ssd

In [7]:
#Convert Monkey video to frames
path_to_video = 'data/monkey_TaskA.mov'
cap = cv2.VideoCapture(path_to_video)
if not cap.isOpened():
    print('{} not opened'.format(path_to_video))
    sys.exit(1)
time_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_counter = 0
frame_num = 0
frame_save_path = './frames/'
create_dir(frame_save_path)

print("Processing...")
while True:
    ret, frame = cap.read()
    if not ret:
        print('Video Ends')
        frame_num = frame_counter - 1
        break
    path = frame_save_path + 'frame{}.tif'.format(frame_counter)
    print('Extracting: %d' % frame_counter)
    clear_output(wait=True)
    cv2.imwrite(path,frame)
    frame_counter += 1
    if cv2.waitKey(30) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Video Ends


In [8]:
#Draw the movements for each frames
frame_processed_path = './processed/'
create_dir(frame_processed_path)
kl = 15
hkl = 7
frame_counter = 0
print("Processing...")
while True:
    img = cv2.imread(frame_save_path + 'frame%d.tif' % frame_counter)
    img2 = cv2.imread(frame_save_path + 'frame%d.tif' % (frame_counter + 1))
    if img2 is None:
        clear_output(wait=True)
        print("No more frames")
        break
    iw = img.shape[1]
    ih = img.shape[0]
    grid_w = iw/kl
    for y in range(50,ih-50,kl):
        for x in range(50,iw-50,kl):
            if img[y,x,0] < 120:
                ssd_min = sys.maxsize
                grid_this = img[x-hkl:x+hkl,y-hkl:y+hkl]
                wanted_x = 0
                wanted_y = 0
                for gby in range(int(y - kl),int(y + kl*2),kl):
                    for gbx in range(int(x - kl),int(y + kl*2),kl):
                        grid_next = img2[gbx-hkl:gbx+hkl,gby-hkl:gby+hkl]
                        now = calc_ssd(grid_next,grid_this)
                        #print("This: ",x,y,gbx,gby,now);
                        if now < ssd_min:
                            ssd_min = now
                            #print(gbx,gby,ssd_min);
                            wanted_x = gbx
                            wanted_y = gby
                if wanted_x != 0 and wanted_y != 0:
                    if x != wanted_x and y != wanted_y:
                        img = arrowdraw(img,x,y,wanted_x,wanted_y)
    output = cv2.imwrite(frame_processed_path + 'prcessed%d.tif' % frame_counter,img)
    clear_output(wait=True)
    print("Generating: %d%%" % int(100*frame_counter/frame_num))
    frame_counter += 1
cv2.destroyAllWindows()
print("Done!") 

No more frames
Done!


In [9]:
count = 0
out = cv2.VideoWriter('./taska.mov', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (int(frame_width), int(frame_height)))
while True:
    img = cv2.imread(frame_processed_path + 'prcessed%d.tif' % count)
    if img is None:
        print('No more frames to be loaded')
        break
    print("Generating Videos: %d%%" % int(100*count/frame_num))
    clear_output(wait=True)
    out.write(img)
    count += 1
out.release()
cv2.destroyAllWindows()
print("Done!")

No more frames to be loaded
Done!
